# Dynamic Programming

## Fibonnacci numbers

[Fibonnacci numbers](https://en.wikipedia.org/wiki/Fibonacci_number) are defined as:  
$F_0 = 0$  
$F_1 = 1$  
$F_n = F_{n-1} + F_{n-2}$ for $n\geq 2$.  
They form a sequence (0,) 1, 1, 2, 3, 5, 8, 13, 21...  
Since their defined recursively, it is only natural to compute them this way!

### Direct recursive implementation

(as seen in previous lectures)

In [1]:
def fib(n):
    if n == 0 or n == 1:
        return n
    else:
        return fib(n-2) + fib(n-1)
    
fib(5)

5

### Recursive implementation with memoisation

In [5]:
def fib_mem(n):
    mem = [None] * (n+1)
    mem[0] = 0
    mem[1] = 1
    return fib_mem_aux(n, mem)
    
def fib_mem_aux(n, mem):
    if mem[n] is None:
        mem[n] =  fib_mem_aux(n-2, mem) + fib_mem_aux(n-1, mem)
    return mem[n]
    
fib_mem(5)

5

### Dynamic Programming (Bottom-up) Computation of mem:

From the previous implementation, it seems clear that $F_n$ can be computed using the list mem[i] for i=0 to n. We can thus write an algorithm that computes this list directly, starting from the base cases, from the bottom to the top:

In [9]:
def fib_dp(n):
    mem = [None] * (n+1)
    mem[0] = 0
    mem[1] = 1
    
    for i in range(2, n+1):
        mem[i] = mem[i-1] + mem[i-2]
    return mem[n]

fib_dp(5)

5

### Testing

We also test the accumulator version seen in a previous lecture:

In [11]:
def fib_acc(n):
    return fib_aux(n, 0, 1)

def fib_aux(n, before_last, last):
    if n == 0:
        return before_last
    else:
        return fib_aux(n-1, last, before_last+last)
    
import timeit
print(timeit.timeit('fib(10)',\
              "from __main__ import fib"))
print(timeit.timeit('fib_mem(10)',\
              "from __main__ import fib_mem"))
print(timeit.timeit('fib_dp(10)',\
              "from __main__ import fib_dp"))
print(timeit.timeit('fib_acc(10)',\
              "from __main__ import fib_acc"))

23.082560774011654
2.976857514004223
1.5252863470086595
1.8258997819939395


## Longuest sequence of positive numbers (LSP)

Given a list of numbers, we want to find the length of the longuest sequence of positive numbers (LSP). For instance, given  

In [49]:
l = [1, 2, 0, -4, 1, 7, 8, 9, 0, 1, 0, 0, -5, -9, -4, 1]

the longuest sequence of positive number is $[1, 7, 8, 9]$ and has length 4.

**To solve a problem using DP, we must identify subproblems such that:**
* One or more subproblems are base cases, which we can solve directly,
* Any other subproblem can be computed using other subproblems that have already been computed,
* The answer to the original problem can be readily retrieved from one or more subproblems.

We can state the subproblem as "Given an index $i \in \{0, \dots, n-1\}$, what is the LSP in the list $l[0:i+1]$ which uses $i$? We denote $L[i]$ the LSP for index $i$.

* A base case is for i=0:
$L[0] =
\begin{cases}
1 \text{ if } l[0] > 0,\\
0 \text{ otherwise.}
\end{cases}$
* The general case for i>0 is:
$L[i] = 
\begin{cases}
1 + L[i-1] \text{ if } l[i] > 0,\\
0 \text{ otherwise.}
\end{cases}$
* The answer to the entire problem is simply the maximum value in $L$. Indeed, the LSP of a sequence is the maximum over all indices $i$ of the longuest sequence which ends at $i$, which we have called $L[i]$.

In Python, this would give us:

In [52]:
def LSP(l):
    """Returns the LSP of the input list."""
    n = len(l)
    L = [None]*n
    for i in range(0, n):
        LSP_aux(l, L, i)
    #print(L)
    return max(L)
    
#We have written this as a function for clarity.
#In practice this would be written directly in the for loop.
def LSP_aux(l, L, i):
    """Computes L[i] using exactly the
       mathematical definition of L given above."""
    if i == 0:
        if l[0] > 0:
            L[0] = 1
        else:
            L[0] = 0
    else:
        if l[i] > 0:
            L[i] = 1 + L[i-1]
        else:
            L[i] = 0

In [53]:
print(LSP(l))
print(LSP(l[:4]))
print(LSP(l[2:4]))
LSP([-1])

4
2
0


0

What is the complexity of our algorithm?

LSP_aux runs in $O(1)$ and thus $LSP$ is clearly $O(n)$.

Extra question: how would you write the above algorithm recursively?

## Maximum subsequence sum (MSS)

In this problem, we wish to find the non-empty subsequence which the highest sum. For instance, given the sequence

In [54]:
l = [1, 2, 0, -4, 1, 2, 96, -1, -100, 99, 0, 0, -5, -9, -7, 25]

the MSS is 103 for the subsequence $[99, 0, 0, -5, -9, -7, 25]$. A naive approach runs in $O(n^3)$ (see slides), which can be improved to $O(n^2)$.

How do we formulate a subproblem to solve the MSS?

* A base case is for i=0:
$L[0] = l[0]$
* The general case for i>0 is:  
$L[i] =\begin{cases}
l[i] + L[i-1] \text{ if } L[i-1] > 0,\\
l[i] \text{ otherwise.}
\end{cases}$  
More simply,
$L[i] = \max(l[i] + L[i-1], l[i])$.
* The answer to the entire problem is simply the maximum value in $L$.

We can implement this directly in Python:

In [55]:
def MSS(l):
    """Returns the MSS of the input list."""
    n = len(l)
    L = [None]*n
    for i in range(0, n):
        MSS_aux(l, L, i)
    #print(L)
    return max(L)
    
#We have written this as a function for clarity.
#In practice this would be written directly in the for loop.
def MSS_aux(l, L, i):
    """Computes L[i] using exactly the
       mathematical definition of L given above."""
    if i == 0:
            L[0] = l[0]
    else:
            L[i] = max(l[i]+L[i-1],l[i])

In [56]:
print(MSS(l))
print(MSS(l[:-1]))
print(MSS([90, 90, 0, -200, 1, 2, 96, -1, -100, 99, 0, 0, -5, -9, -7, 25]))

103
99
180
